## 🚀 Quick Start Guide

**Before running this notebook:**

1. **Get a Gemini API Key**
   - Visit [Google AI Studio](https://aistudio.google.com/app/api-keys)
   - Click "Create API Key"
   - Copy your API key

2. **Create `.env` file**
   - In this directory (`/code/`), create a file named `.env`
   - Add the following line:
   ```
   GOOGLE_API_KEY=your_actual_api_key_here
   ```
   - Save the file

3. **Run the notebook**
   - Execute cells in order from top to bottom
   - The system will load your API key automatically
   - Interactive HITL checkpoints will prompt for approval/rejection
   - MCP Python Refactoring provides professional-grade analysis


**Note:** A `.env.example` file is provided as a template.


# ARC-DSL Type Annotation Agent - Phase 1

**Project:** ARC-DSL Refactoring Agent System  
**Track:** Kaggle Agents Intensive - Freestyle  
**Date:** November 24, 2025

## Overview

A human-in-the-loop (HITL) agent system that incrementally adds type annotations to the [arc-dsl codebase](https://github.com/michaelhodel/arc-dsl) through intelligent analysis and validation.

**Core Philosophy:** Humans approve strategy, agents execute tactics.

### Phase 1: Type Annotation System

Phase 1 focuses on adding type hints to 400+ solver functions in `solvers.py` using the strict ARC-DSL type system (Grid, Piece, Object, Objects, Indices, Integer, IntegerSet).

**Key Features:**
- **Specialized Agent:** Type annotation expert with domain-specific knowledge
- **Automated Analysis:** External analyzer script (`analyze_solver_types.py`) infers types
- **HITL Workflow:** Interactive approval with refine/skip/abort options
- **Automatic Testing:** Runs `tests.py` and `main.py` after each approval
- **Auto-Rollback:** Restores original code if tests fail (regression prevention)
- **Progress Tracking:** Shows annotation progress across the codebase

### Current Status

- **Implementation:** ✅ Complete and tested
- **Solvers Processed:** 10 test cases (8 approved, 1 refined, 2 skipped)
- **Test Results:** All regression tests passing
- **Next Steps:** Scale to full 400+ solver functions

### Refactoring Goals

1. **Add Type Annotations:** Eliminate type ambiguity with explicit type hints
2. **Maintain Compatibility:** Zero regressions (tests.py must pass, main.py ≥390/1000)
3. **Incremental Progress:** One function at a time with HITL validation

## Section 1: Import Libraries

Import all necessary libraries for the multi-agent system.

In [1]:
# Install required packages
%pip install -q python-dotenv google-genai google-adk ipywidgets

# Clone arc-dsl repository if not already present
import os
if not os.path.exists('arc-dsl'):
    !git clone https://github.com/michaelhodel/arc-dsl.git
    print("✓ arc-dsl repository cloned")
else:
    print("✓ arc-dsl repository already exists")

Note: you may need to restart the kernel to use updated packages.
✓ arc-dsl repository already exists


## Section 2: Configure Gemini API Key

Load the Gemini API key from the .env file.

In [2]:
import os
from dotenv import load_dotenv

try:
    # Load environment variables from .env file
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}"
    )

✅ Gemini API key setup complete.


In [3]:
# Configure Gemini API
from google import genai
from google.genai import types

MODEL_NAME = 'gemini-2.0-flash-exp'  # Using Gemini 2.0 Flash

# Initialize Gemini client
client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))

# Configure retry options for handling transient errors (429, 500, 503, 504)
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Exponential backoff multiplier (delay grows: 1s, 7s, 49s...)
    initial_delay=1,  # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

print("✅ Gemini client configured")
print(f"   Model: {MODEL_NAME}")
print(f"   Retry configuration: {retry_config.attempts} attempts")
print(f"   Exponential backoff: 1s → 7s → 49s → 343s → 2401s")
print(f"   Retry on HTTP: {retry_config.http_status_codes}")

✅ Gemini client configured
   Model: gemini-2.0-flash-exp
   Retry configuration: 5 attempts
   Exponential backoff: 1s → 7s → 49s → 343s → 2401s
   Retry on HTTP: [429, 500, 503, 504]


## Section 3: Configure Gemini Client

Initialize the Gemini client with retry configuration for the agent system.

## Section 4: Agent Base Class

Define the base RefactoringAgent class used by the type annotation system.

In [4]:
# Agent Base Class Implementation
import time
from typing import Dict

class RefactoringAgent:
    """Base class for refactoring agents with retry logic"""
    
    def __init__(self, name: str, system_prompt: str):
        self.name = name
        self.system_prompt = system_prompt
        self.client = client
        self.model_name = MODEL_NAME
        self.retry_config = retry_config
    
    def call(self, prompt: str, context: Dict = None) -> str:
        """Call agent with prompt and context, with automatic retry on HTTP errors"""
        full_prompt = f"{self.system_prompt}\n\n{prompt}"
        
        if context:
            import json
            full_prompt += f"\n\nContext:\n{json.dumps(context, indent=2)}"
        
        # Retry logic with exponential backoff
        max_attempts = self.retry_config.attempts
        initial_delay = self.retry_config.initial_delay
        exp_base = self.retry_config.exp_base
        retry_codes = self.retry_config.http_status_codes
        
        for attempt in range(max_attempts):
            try:
                response = self.client.models.generate_content(
                    model=self.model_name,
                    contents=full_prompt
                )
                return response.text
            except Exception as e:
                error_str = str(e)
                
                # Check if this is a retryable HTTP error
                is_retryable = any(str(code) in error_str for code in retry_codes)
                is_last_attempt = (attempt == max_attempts - 1)
                
                if is_retryable and not is_last_attempt:
                    # Calculate delay with exponential backoff
                    delay = initial_delay * (exp_base ** attempt)
                    print(f"⚠️  {self.name}: HTTP error on attempt {attempt + 1}/{max_attempts}")
                    print(f"   Retrying in {delay:.1f}s... (exponential backoff)")
                    time.sleep(delay)
                else:
                    # Non-retryable error or final attempt
                    raise Exception(f"{self.name} failed: {error_str}")
        
        raise Exception(f"{self.name}: Exhausted all {max_attempts} retry attempts")

print("✓ RefactoringAgent base class defined")

✓ RefactoringAgent base class defined


## Section 5: Phase 1 - Type Annotation System

Implement the HITL type annotation workflow with automated testing and rollback.

In [5]:
# Setup environment for Phase 1
import subprocess
import re
import ast
from pathlib import Path
import os

# Ensure we're in the correct directory
os.chdir('/Users/pierre/Library/CloudStorage/GoogleDrive-pierre@baume.org/My Drive/AI Agents Intensive/code')

print("✓ Phase 1 Environment Ready")
print(f"  Working Directory: {os.getcwd()}")
print(f"  Analyzer Tool: analyze_solver_types.py")
print(f"  Target File: arc-dsl/solvers.py")

✓ Phase 1 Environment Ready
  Working Directory: /Users/pierre/Library/CloudStorage/GoogleDrive-pierre@baume.org/My Drive/AI Agents Intensive/code
  Analyzer Tool: analyze_solver_types.py
  Target File: arc-dsl/solvers.py


In [6]:
def analyze_solver_types_tool(solver_name: str) -> dict:
    """
    Analyze a solver function and return type annotations.
    
    Args:
        solver_name: Name of solver function (e.g., 'solve_67a3c6ac')
    
    Returns:
        dict with 'variables', 'annotated_code', 'success' keys
    """
    try:
        # Run the analyzer
        result = subprocess.run(
            ['python', 'analyze_solver_types.py', solver_name],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if result.returncode != 0:
            return {
                'success': False,
                'error': result.stderr,
                'solver': solver_name
            }
        
        # Parse output to extract variable types and annotated code
        output = result.stdout
        lines = output.split('\n')
        
        # Extract variables section
        variables = {}
        in_variables = False
        for line in lines:
            if line.startswith('Variables ('):
                in_variables = True
                continue
            elif in_variables:
                if line.startswith('Has Callables:') or line.strip() == '':
                    break
                # Variable lines are indented and have format "  var: type"
                if ':' in line and line.startswith('  '):
                    parts = line.strip().split(':', 1)
                    if len(parts) == 2:
                        var, vtype = parts
                        variables[var.strip()] = vtype.strip()
        
        # Extract annotated code section
        annotated_code = ""
        in_code = False
        skip_next_separator = False
        
        for i, line in enumerate(lines):
            if 'Generated Annotated Code:' in line:
                in_code = True
                skip_next_separator = True
                continue
            elif in_code:
                # Skip the separator line (====)
                if skip_next_separator and line.startswith('===='):
                    skip_next_separator = False
                    continue
                # Stop at next separator or HITL section
                if line.startswith('====') or 'HITL Refactoring Script Info:' in line:
                    break
                # Collect code lines
                annotated_code += line + '\n'
        
        return {
            'success': True,
            'solver': solver_name,
            'variables': variables,
            'annotated_code': annotated_code.strip(),
            'variable_count': len(variables)
        }
        
    except subprocess.TimeoutExpired:
        return {
            'success': False,
            'error': 'Analysis timed out',
            'solver': solver_name
        }
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'solver': solver_name
        }


def get_annotation_progress_tool() -> dict:
    """
    Count how many solvers have been annotated with type hints.
    
    Returns:
        dict with 'total', 'annotated', 'remaining', 'percent' keys
    """
    try:
        with open('arc-dsl/solvers.py', 'r') as f:
            content = f.read()
        
        # Count all solver functions
        total = len(re.findall(r'^def solve_\w+\(', content, re.MULTILINE))
        
        # Count annotated functions (have type hints)
        annotated = len(re.findall(r'^def solve_\w+\(I: Grid\) -> Grid:', content, re.MULTILINE))
        
        remaining = total - annotated
        percent = round((annotated / total * 100), 1) if total > 0 else 0
        
        return {
            'total': total,
            'annotated': annotated,
            'remaining': remaining,
            'percent': percent
        }
    except Exception as e:
        return {
            'total': 0,
            'annotated': 0,
            'remaining': 0,
            'percent': 0,
            'error': str(e)
        }


def get_next_batch_tool(batch_size: int = 10, start_line: int = 1) -> dict:
    """
    Get next batch of unannotated solver functions.
    
    Args:
        batch_size: Number of solvers to return
        start_line: Starting line number to search from
    
    Returns:
        dict with 'batch' (list of solver names) and 'line_ranges' (dict of {name: (start, end)})
    """
    try:
        with open('arc-dsl/solvers.py', 'r') as f:
            lines = f.readlines()
        
        batch = []
        line_ranges = {}
        
        i = max(0, start_line - 1)  # Convert to 0-indexed
        while i < len(lines) and len(batch) < batch_size:
            line = lines[i]
            
            # Look for unannotated solver functions
            match = re.match(r'^def (solve_\w+)\(I\):', line)
            if match:
                solver_name = match.group(1)
                start = i + 1  # Convert back to 1-indexed
                
                # Find end of function (next def or end of file)
                end = start
                for j in range(i + 1, len(lines)):
                    if lines[j].startswith('def '):
                        end = j  # Line before next function
                        break
                else:
                    end = len(lines)
                
                batch.append(solver_name)
                line_ranges[solver_name] = (start, end)
            
            i += 1
        
        return {
            'batch': batch,
            'line_ranges': line_ranges
        }
    except Exception as e:
        return {
            'batch': [],
            'line_ranges': {},
            'error': str(e)
        }

print("✓ Phase 1 Tools Defined")

✓ Phase 1 Tools Defined


In [7]:
# Create Type Annotation Agent
type_annotation_agent = RefactoringAgent(
    name="Type Annotation Agent",
    system_prompt="""You are a specialized Python type annotation expert for the ARC-DSL project.

Your task: Add type hints to solver functions using the strict type system:
- Grid (2D color grid)
- Piece (region of grid)
- Object (connected component)
- Objects (collection of Objects)
- Indices (set of cell positions)
- Integer (color or count)
- IntegerSet (set of integers)

CRITICAL RULES:
1. ONLY add type annotations - DO NOT rewrite logic
2. Keep every line of code exactly as-is
3. Function signature: def solve_X(I: Grid) -> Grid:
4. Variable annotations: var: Type = expression
5. Use the EXACT variable names from the original code
6. Preserve all whitespace and formatting
7. Use inferred types from analyze_solver_types.py output
8. Return complete annotated function
9. No explanations - only code

Example transformation:
BEFORE:
def solve_67a3c6ac(I):
    O = vmirror(I)
    return O

AFTER:
def solve_67a3c6ac(I: Grid) -> Grid:
    O: Grid = vmirror(I)
    return O
"""
)

print("✓ Type Annotation Agent Created")
print(f"  Agent name: {type_annotation_agent.name}")
print(f"  Model: {type_annotation_agent.model_name}")

✓ Type Annotation Agent Created
  Agent name: Type Annotation Agent
  Model: gemini-2.0-flash-exp


In [ ]:
import ast
import time
from IPython.display import clear_output

def run_type_annotation_test(batch_size: int = 10, start_line: int = 1, end_line: int = None):
    """
    Interactive workflow for type annotation with human approval.
    Uses clear_output to maintain a clean interface and avoid duplication artifacts.
    
    Args:
        batch_size: Number of solvers to process
        start_line: Starting line in solvers.py
        end_line: Ending line (None = search to end of file)
    """
    # Use the pre-created type_annotation_agent (defined in previous cell)
    # Note: Each call to agent.call() is stateless - no conversation history contamination
    
    # Get initial progress
    progress = get_annotation_progress_tool()
    
    # Get batch of unannotated solvers
    batch_info = get_next_batch_tool(batch_size, start_line)
    batch = batch_info['batch']
    line_ranges = batch_info['line_ranges']
    
    # Filter by end_line if specified
    if end_line is not None:
        batch = [s for s in batch if line_ranges[s][0] <= end_line]
    
    print(f"\n🎯 Processing batch of {len(batch)} solvers (lines {start_line}-{end_line or 'EOF'})")
    print(f"   Solvers: {', '.join(batch[:5])}{'...' if len(batch) > 5 else ''}\n")
    
    results = {
        'processed': 0,
        'approved': 0,
        'refined': 0,
        'skipped': 0,
        'failed': 0,
        'details': []
    }
    
    for i, solver_name in enumerate(batch):
        clear_output(wait=True)
        print(f"\n📊 Batch Progress: {results['processed']}/{len(batch)} processed")
        print(f"   Approved: {results['approved']} | Refined: {results['refined']} | Skipped: {results['skipped']} | Failed: {results['failed']}")
        
        print(f"\n{'='*70}")
        print(f"[{i}/{len(batch)}] Processing: {solver_name}")
        print(f"{'='*70}")
        
        # Analyze the solver
        analysis = analyze_solver_types_tool(solver_name)
        
        if not analysis['success']:
            print(f"❌ Analysis failed: {analysis.get('error', 'Unknown error')}")
            results['failed'] += 1
            results['details'].append({
                'solver': solver_name,
                'status': 'failed',
                'error': analysis.get('error')
            })
            time.sleep(2) # Pause to let user see error
            continue
        
        # Check if analysis returned valid results
        if not analysis.get('annotated_code') or analysis.get('annotated_code', '').strip() == '':
            print(f"⚠️  Analysis returned empty annotated code - skipping")
            results['failed'] += 1
            results['details'].append({
                'solver': solver_name,
                'status': 'failed',
                'error': 'Empty annotated code from analyzer'
            })
            time.sleep(2)
            continue
        
        # Display analysis results
        print(f"\n📋 Analysis Results:")
        print(f"   Variables: {analysis['variable_count']}")
        for var, vtype in list(analysis['variables'].items())[:10]:
            print(f"      {var}: {vtype}")
        if len(analysis['variables']) > 10:
            print(f"      ... and {len(analysis['variables']) - 10} more")
        
        # Show proposed annotated code
        print(f"\n📝 Proposed Annotated Code:")
        print("─" * 70)
        print(analysis['annotated_code'][:500])
        if len(analysis['annotated_code']) > 500:
            print("...")
        print("─" * 70)
        
        # Human-in-the-loop decision
        max_refinement_attempts = 3
        refinement_count = 0
        
        while refinement_count < max_refinement_attempts:
            decision = input(f"\n❓ Decision [a=approve, r=refine, s=skip, q=abort]: ").strip().lower()
            
            if decision == 'q':
                print("⛔ Workflow aborted by user")
                return results
            
            elif decision == 's':
                print(f"⏭️  Skipping {solver_name}")
                results['skipped'] += 1
                results['details'].append({
                    'solver': solver_name,
                    'status': 'skipped'
                })
                time.sleep(1) # Add delay to see skip message
                break
            
            elif decision == 'r':
                refinement_count += 1
                print(f"\n🔄 Refinement attempt {refinement_count}/{max_refinement_attempts}")
                
                feedback = input("   Provide refinement instructions: ").strip()
                
                # Use the agent to refine based on feedback
                refinement_prompt = f"""
Based on this feedback: "{feedback}"

Re-annotate this function:
{analysis['annotated_code']}

Apply the requested refinements while following all type annotation rules.
"""
                
                # Call the pre-created agent (each call is stateless)
                response = type_annotation_agent.call(refinement_prompt)
                
                print(f"\n📝 Refined Code:")
                print("─" * 70)
                print(response[:500])
                if len(response) > 500:
                    print("...")
                print("─" * 70)
                
                # Update the proposal
                analysis['annotated_code'] = response
                
                continue  # Ask for decision again
            
            elif decision == 'a':
                print(f"\n✅ Approved: {solver_name}")
                
                # Double-check we have valid code before applying
                if not analysis['annotated_code'] or analysis['annotated_code'].strip() == '':
                    print(f"   ❌ Cannot apply: annotated code is empty!")
                    results['failed'] += 1
                    results['details'].append({
                        'solver': solver_name,
                        'status': 'failed',
                        'error': 'Empty annotated code'
                    })
                    break
                
                # Apply the annotation using AST-based replacement
                try:
                    with open('arc-dsl/solvers.py', 'r') as f:
                        file_content = f.read()
                    
                    # Use AST to find exact function boundaries
                    tree = ast.parse(file_content)
                    target_func = None
                    
                    for node in ast.walk(tree):
                        if isinstance(node, ast.FunctionDef) and node.name == solver_name:
                            target_func = node
                            break
                    
                    if target_func is None:
                        print(f"   ⚠️  Function {solver_name} not found in AST")
                        results['failed'] += 1
                        results['details'].append({
                            'solver': solver_name,
                            'status': 'failed',
                            'error': 'Function not found in AST'
                        })
                        break
                    
                    # Get exact line boundaries (1-indexed)
                    start_line_idx = target_func.lineno - 1  # Convert to 0-indexed
                    end_line_idx = target_func.end_lineno  # This is already the correct end
                    
                    # Split content into lines
                    lines = file_content.split('\n')
                    
                    # Replace the function (no extra empty line)
                    new_lines = (
                        lines[:start_line_idx] +
                        [analysis['annotated_code']] +
                        lines[end_line_idx:]
                    )
                    
                    new_content = '\n'.join(new_lines)
                    
                    # Write back
                    with open('arc-dsl/solvers.py', 'w') as f:
                        f.write(new_content)
                    
                    print(f"   ✓ Type annotations applied successfully", flush=True)
                    
                    # Run regression tests
                    print(f"\n🧪 Running regression tests...", flush=True)
                    
                    # Test 1: Run tests.py
                    test_result = subprocess.run(
                        ['python', 'tests.py'],
                        cwd='arc-dsl',
                        capture_output=True,
                        text=True,
                        timeout=30
                    )
                    
                    tests_passed = test_result.returncode == 0
                    
                    if tests_passed:
                        print(f"   ✅ tests.py passed")
                    else:
                        print(f"   ❌ tests.py failed")
                        if test_result.stderr:
                            print(f"      Error: {test_result.stderr[:200]}")
                    
                    # Test 2: Run main.py and check solver count
                    main_result = subprocess.run(
                        ['python', 'main.py'],
                        cwd='arc-dsl',
                        capture_output=True,
                        text=True,
                        timeout=60
                    )
                    
                    solved_count = None
                    if main_result.returncode == 0 and main_result.stdout:
                        # Parse "Tasks solved correctly: 390 out of 1000"
                        import re
                        match = re.search(r'(\d+)\s+out of\s+(\d+)', main_result.stdout)
                        if match:
                            solved_count = int(match.group(1))
                            expected_min = 390  # Baseline from initial run
                            
                            if solved_count >= expected_min:
                                print(f"   ✅ main.py passed ({solved_count}/1000 tasks solved)")
                            else:
                                print(f"   ❌ main.py regression ({solved_count}/1000, expected ≥{expected_min})")
                                tests_passed = False
                    else:
                        print(f"   ⚠️  main.py couldn't be verified")
                        # Don't fail on main.py verification issues
                    
                    # Decision based on test results
                    if tests_passed:
                        print(f"\n   ✅ All regression checks passed - changes kept")
                        
                        results['approved'] += 1
                        if refinement_count > 0:
                            results['refined'] += 1
                        
                        results['details'].append({
                            'solver': solver_name,
                            'status': 'approved',
                            'variables': analysis['variable_count'],
                            'refinements': refinement_count,
                            'tests': 'passed',
                            'solved_count': solved_count
                        })
                        time.sleep(1) # Pause to let user see success
                        break  # Exit decision loop after successful approval
                    else:
                        print(f"\n   ❌ REGRESSION DETECTED - rolling back changes")
                        
                        # Restore original content
                        with open('arc-dsl/solvers.py', 'w') as f:
                            f.write(file_content)
                        print(f"   ↩️  Original code restored")
                        
                        results['failed'] += 1
                        results['details'].append({
                            'solver': solver_name,
                            'status': 'failed',
                            'error': 'Regression: tests failed',
                            'test_output': test_result.stderr[:500] if test_result.stderr else test_result.stdout[:500],
                            'solved_count': solved_count
                        })
                        time.sleep(3) # Pause to let user see failure
                        break  # Exit decision loop after failed test
                    
                except Exception as e:
                    print(f"   ❌ Failed to apply annotations: {e}")
                    results['failed'] += 1
                    results['details'].append({
                        'solver': solver_name,
                        'status': 'failed',
                        'error': str(e)
                    })
                    time.sleep(2)
                
                break
            
            else:
                print("❌ Invalid choice. Please enter 'a', 'r', 's', or 'q'")
        
        results['processed'] += 1
    
    print(f"\n\n{'='*70}")
    print("📊 WORKFLOW SUMMARY")
    print(f"{'='*70}")
    print(f"Processed:  {results['processed']}/{len(batch)} solvers")
    print(f"✅ Approved: {results['approved']}")
    print(f"🔄 Refined:  {results['refined']}")
    print(f"⏭️  Skipped:  {results['skipped']}")
    print(f"❌ Failed:   {results['failed']}")
    print(f"\nOverall Progress:")
    print(f"   Annotated: {final_progress['annotated']}/{final_progress['total']} ({final_progress['percent']}%)")
    print(f"   Remaining: {final_progress['remaining']}")
    print(f"{'='*70}\n")
    
    return results

print("✓ Phase 1 Workflow Function Ready")

✓ Phase 1 Workflow Function Ready


In [9]:
# Run Phase 1 Type Annotation Test
# Process small batch for testing (lines 5-73 contain ~10 solvers)

results = run_type_annotation_test(
    batch_size=10,
    start_line=5,
    end_line=300
)


📊 Batch Progress: 1/10 processed
   Approved: 1 | Refined: 0 | Skipped: 0 | Failed: 0

[1/10] Processing: solve_25ff71a9

📋 Analysis Results:
   Variables: 4
      I: Grid
      x1: Objects
      x2: Any
      O: Grid

📝 Proposed Annotated Code:
──────────────────────────────────────────────────────────────────────
def solve_25ff71a9(I: Grid) -> Grid:
    x1: Objects = objects(I, T, T, T)
    x2: Any = first(x1)
    O: Grid = move(I, x2, DOWN)
    return O
──────────────────────────────────────────────────────────────────────
⛔ Workflow aborted by user
⛔ Workflow aborted by user
